In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
df = pd.read_csv("encoded_clean_clinical_wav2vec2.csv")

In [5]:
df.head()

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47,feat_48,feat_49,feat_50,feat_51,feat_52,feat_53,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,feat_94,feat_95,feat_96,feat_97,feat_98,feat_99,feat_100,feat_101,feat_102,feat_103,feat_104,feat_105,feat_106,feat_107,feat_108,feat_109,feat_110,feat_111,feat_112,feat_113,feat_114,feat_115,feat_116,feat_117,feat_118,feat_119,feat_120,feat_121,feat_122,feat_123,feat_124,feat_125,feat_126,feat_127,feat_128,feat_129,feat_130,feat_131,feat_132,feat_133,feat_134,feat_135,feat_136,feat_137,feat_138,feat_139,feat_140,feat_141,feat_142,feat_143,feat_144,feat_145,feat_146,feat_147,feat_148,feat_149,feat_150,feat_151,feat_152,feat_153,feat_154,feat_155,feat_156,feat_157,feat_158,feat_159,feat_160,feat_161,feat_162,feat_163,feat_164,feat_165,feat_166,feat_167,feat_168,feat_169,feat_170,feat_171,feat_172,feat_173,feat_174,feat_175,feat_176,feat_177,feat_178,feat_179,feat_180,feat_181,feat_182,feat_183,feat_184,feat_185,feat_186,feat_187,feat_188,feat_189,feat_190,feat_191,feat_192,feat_193,feat_194,feat_195,feat_196,feat_197,feat_198,feat_199,feat_200,feat_201,feat_202,feat_203,feat_204,feat_205,feat_206,feat_207,feat_208,feat_209,feat_210,feat_211,feat_212,feat_213,feat_214,feat_215,feat_216,feat_217,feat_218,feat_219,feat_220,feat_221,feat_222,feat_223,feat_224,feat_225,feat_226,feat_227,feat_228,feat_229,feat_230,feat_231,feat_232,feat_233,feat_234,feat_235,feat_236,feat_237,feat_238,feat_239,feat_240,feat_241,feat_242,feat_243,feat_244,feat_245,feat_246,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,feat_257,feat_258,feat_259,feat_260,feat_261,feat_262,feat_263,feat_264,feat_265,feat_266,feat_267,feat_268,feat_269,feat_270,feat_271,feat_272,feat_273,feat_274,feat_275,feat_276,feat_277,feat_278,feat_279,feat_280,feat_281,feat_282,feat_283,feat_284,feat_285,feat_286,feat_287,feat_288,feat_289,feat_290,feat_291,feat_292,feat_293,feat_294,feat_295,feat_296,feat_297,feat_298,feat_299,feat_300,feat_301,feat_302,feat_303,feat_304,feat_305,feat_306,feat_307,feat_308,feat_309,feat_310,feat_311,feat_312,feat_313,feat_314,feat_315,feat_316,feat_317,feat_318,feat_319,feat_320,feat_321,feat_322,feat_323,feat_324,feat_325,feat_326,feat_327,feat_328,feat_329,feat_330,feat_331,feat_332,feat_333,feat_334,feat_335,feat_336,feat_337,feat_338,feat_339,feat_340,feat_341,feat_342,feat_343,feat_344,feat_345,feat_346,feat_347,feat_348,feat_349,feat_350,feat_351,feat_352,feat_353,feat_354,feat_355,feat_356,feat_357,feat_358,feat_359,feat_360,feat_361,feat_362,feat_363,feat_364,feat_365,feat_366,feat_367,feat_368,feat_369,feat_370,feat_371,feat_372,feat_373,feat_374,feat_375,feat_376,feat_377,feat_378,feat_379,feat_380,feat_381,feat_382,feat_383,feat_384,feat_385,feat_386,feat_387,feat_388,feat_389,feat_390,feat_391,feat_392,feat_393,feat_394,feat_395,feat_396,feat_397,feat_398,feat_399,feat_400,feat_401,feat_402,feat_403,feat_404,feat_405,feat_406,feat_407,feat_408,feat_409,feat_410,feat_411,feat_412,feat_413,feat_414,feat_415,feat_416,feat_417,feat_418,feat_419,feat_420,feat_421,feat_422,feat_423,feat_424,feat_425,feat_426,feat_427,feat_428,feat_429,feat_430,feat_431,feat_432,feat_433,feat_434,feat_435,feat_436,feat_437,feat_438,feat_439,feat_440,feat_441,feat_442,feat_443,feat_444,feat_445,feat_446,feat_447,feat_448,feat_449,feat_450,feat_451,feat_452,feat_453,feat_454,feat_455,feat_

In [6]:
df.columns

Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9',
       ...
       'tb_prior', 'tb_prior_Pul', 'hemoptysis', 'heart_rate', 'temperature',
       'weight_loss', 'smoke_lweek', 'fever', 'night_sweats', 'tb_status'],
      dtype='object', length=783)

In [8]:
X = df.drop('tb_status', axis=1)
y = df['tb_status']

In [9]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
# Split into training and validation (optional but good for evaluation)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [14]:
print("Validation Score:", model.score(X_val, y_val))

Validation Score: 0.9980732177263969


In [34]:
df_test = pd.read_csv('solicited_test.csv')

In [35]:
df_test.head()

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47,feat_48,feat_49,feat_50,feat_51,feat_52,feat_53,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,feat_94,feat_95,feat_96,feat_97,feat_98,feat_99,feat_100,feat_101,feat_102,feat_103,feat_104,feat_105,feat_106,feat_107,feat_108,feat_109,feat_110,feat_111,feat_112,feat_113,feat_114,feat_115,feat_116,feat_117,feat_118,feat_119,feat_120,feat_121,feat_122,feat_123,feat_124,feat_125,feat_126,feat_127,feat_128,feat_129,feat_130,feat_131,feat_132,feat_133,feat_134,feat_135,feat_136,feat_137,feat_138,feat_139,feat_140,feat_141,feat_142,feat_143,feat_144,feat_145,feat_146,feat_147,feat_148,feat_149,feat_150,feat_151,feat_152,feat_153,feat_154,feat_155,feat_156,feat_157,feat_158,feat_159,feat_160,feat_161,feat_162,feat_163,feat_164,feat_165,feat_166,feat_167,feat_168,feat_169,feat_170,feat_171,feat_172,feat_173,feat_174,feat_175,feat_176,feat_177,feat_178,feat_179,feat_180,feat_181,feat_182,feat_183,feat_184,feat_185,feat_186,feat_187,feat_188,feat_189,feat_190,feat_191,feat_192,feat_193,feat_194,feat_195,feat_196,feat_197,feat_198,feat_199,feat_200,feat_201,feat_202,feat_203,feat_204,feat_205,feat_206,feat_207,feat_208,feat_209,feat_210,feat_211,feat_212,feat_213,feat_214,feat_215,feat_216,feat_217,feat_218,feat_219,feat_220,feat_221,feat_222,feat_223,feat_224,feat_225,feat_226,feat_227,feat_228,feat_229,feat_230,feat_231,feat_232,feat_233,feat_234,feat_235,feat_236,feat_237,feat_238,feat_239,feat_240,feat_241,feat_242,feat_243,feat_244,feat_245,feat_246,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,feat_257,feat_258,feat_259,feat_260,feat_261,feat_262,feat_263,feat_264,feat_265,feat_266,feat_267,feat_268,feat_269,feat_270,feat_271,feat_272,feat_273,feat_274,feat_275,feat_276,feat_277,feat_278,feat_279,feat_280,feat_281,feat_282,feat_283,feat_284,feat_285,feat_286,feat_287,feat_288,feat_289,feat_290,feat_291,feat_292,feat_293,feat_294,feat_295,feat_296,feat_297,feat_298,feat_299,feat_300,feat_301,feat_302,feat_303,feat_304,feat_305,feat_306,feat_307,feat_308,feat_309,feat_310,feat_311,feat_312,feat_313,feat_314,feat_315,feat_316,feat_317,feat_318,feat_319,feat_320,feat_321,feat_322,feat_323,feat_324,feat_325,feat_326,feat_327,feat_328,feat_329,feat_330,feat_331,feat_332,feat_333,feat_334,feat_335,feat_336,feat_337,feat_338,feat_339,feat_340,feat_341,feat_342,feat_343,feat_344,feat_345,feat_346,feat_347,feat_348,feat_349,feat_350,feat_351,feat_352,feat_353,feat_354,feat_355,feat_356,feat_357,feat_358,feat_359,feat_360,feat_361,feat_362,feat_363,feat_364,feat_365,feat_366,feat_367,feat_368,feat_369,feat_370,feat_371,feat_372,feat_373,feat_374,feat_375,feat_376,feat_377,feat_378,feat_379,feat_380,feat_381,feat_382,feat_383,feat_384,feat_385,feat_386,feat_387,feat_388,feat_389,feat_390,feat_391,feat_392,feat_393,feat_394,feat_395,feat_396,feat_397,feat_398,feat_399,feat_400,feat_401,feat_402,feat_403,feat_404,feat_405,feat_406,feat_407,feat_408,feat_409,feat_410,feat_411,feat_412,feat_413,feat_414,feat_415,feat_416,feat_417,feat_418,feat_419,feat_420,feat_421,feat_422,feat_423,feat_424,feat_425,feat_426,feat_427,feat_428,feat_429,feat_430,feat_431,feat_432,feat_433,feat_434,feat_435,feat_436,feat_437,feat_438,feat_439,feat_440,feat_441,feat_442,feat_443,feat_444,feat_445,feat_446,feat_447,feat_448,feat_449,feat_450,feat_451,feat_452,feat_453,feat_454,feat_455,feat_

In [55]:
# Replace Yes/No/Not sure with 1/0 and Male/Female with 1/0
df_test.replace({
    'Yes': 1,
    'No': 0,
    'Not sure': 0,
    'Male': 1,
    'Female': 0
}, inplace=True)

In [82]:
df_test.columns

Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9',
       ...
       'tb_prior', 'tb_prior_Pul', 'hemoptysis', 'heart_rate', 'temperature',
       'weight_loss', 'smoke_lweek', 'fever', 'night_sweats',
       'predicted_tb_status'],
      dtype='object', length=785)

In [57]:
df_test.head()

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,feat_19,feat_20,feat_21,feat_22,feat_23,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,feat_33,feat_34,feat_35,feat_36,feat_37,feat_38,feat_39,feat_40,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47,feat_48,feat_49,feat_50,feat_51,feat_52,feat_53,feat_54,feat_55,feat_56,feat_57,feat_58,feat_59,feat_60,feat_61,feat_62,feat_63,feat_64,feat_65,feat_66,feat_67,feat_68,feat_69,feat_70,feat_71,feat_72,feat_73,feat_74,feat_75,feat_76,feat_77,feat_78,feat_79,feat_80,feat_81,feat_82,feat_83,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,feat_94,feat_95,feat_96,feat_97,feat_98,feat_99,feat_100,feat_101,feat_102,feat_103,feat_104,feat_105,feat_106,feat_107,feat_108,feat_109,feat_110,feat_111,feat_112,feat_113,feat_114,feat_115,feat_116,feat_117,feat_118,feat_119,feat_120,feat_121,feat_122,feat_123,feat_124,feat_125,feat_126,feat_127,feat_128,feat_129,feat_130,feat_131,feat_132,feat_133,feat_134,feat_135,feat_136,feat_137,feat_138,feat_139,feat_140,feat_141,feat_142,feat_143,feat_144,feat_145,feat_146,feat_147,feat_148,feat_149,feat_150,feat_151,feat_152,feat_153,feat_154,feat_155,feat_156,feat_157,feat_158,feat_159,feat_160,feat_161,feat_162,feat_163,feat_164,feat_165,feat_166,feat_167,feat_168,feat_169,feat_170,feat_171,feat_172,feat_173,feat_174,feat_175,feat_176,feat_177,feat_178,feat_179,feat_180,feat_181,feat_182,feat_183,feat_184,feat_185,feat_186,feat_187,feat_188,feat_189,feat_190,feat_191,feat_192,feat_193,feat_194,feat_195,feat_196,feat_197,feat_198,feat_199,feat_200,feat_201,feat_202,feat_203,feat_204,feat_205,feat_206,feat_207,feat_208,feat_209,feat_210,feat_211,feat_212,feat_213,feat_214,feat_215,feat_216,feat_217,feat_218,feat_219,feat_220,feat_221,feat_222,feat_223,feat_224,feat_225,feat_226,feat_227,feat_228,feat_229,feat_230,feat_231,feat_232,feat_233,feat_234,feat_235,feat_236,feat_237,feat_238,feat_239,feat_240,feat_241,feat_242,feat_243,feat_244,feat_245,feat_246,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,feat_257,feat_258,feat_259,feat_260,feat_261,feat_262,feat_263,feat_264,feat_265,feat_266,feat_267,feat_268,feat_269,feat_270,feat_271,feat_272,feat_273,feat_274,feat_275,feat_276,feat_277,feat_278,feat_279,feat_280,feat_281,feat_282,feat_283,feat_284,feat_285,feat_286,feat_287,feat_288,feat_289,feat_290,feat_291,feat_292,feat_293,feat_294,feat_295,feat_296,feat_297,feat_298,feat_299,feat_300,feat_301,feat_302,feat_303,feat_304,feat_305,feat_306,feat_307,feat_308,feat_309,feat_310,feat_311,feat_312,feat_313,feat_314,feat_315,feat_316,feat_317,feat_318,feat_319,feat_320,feat_321,feat_322,feat_323,feat_324,feat_325,feat_326,feat_327,feat_328,feat_329,feat_330,feat_331,feat_332,feat_333,feat_334,feat_335,feat_336,feat_337,feat_338,feat_339,feat_340,feat_341,feat_342,feat_343,feat_344,feat_345,feat_346,feat_347,feat_348,feat_349,feat_350,feat_351,feat_352,feat_353,feat_354,feat_355,feat_356,feat_357,feat_358,feat_359,feat_360,feat_361,feat_362,feat_363,feat_364,feat_365,feat_366,feat_367,feat_368,feat_369,feat_370,feat_371,feat_372,feat_373,feat_374,feat_375,feat_376,feat_377,feat_378,feat_379,feat_380,feat_381,feat_382,feat_383,feat_384,feat_385,feat_386,feat_387,feat_388,feat_389,feat_390,feat_391,feat_392,feat_393,feat_394,feat_395,feat_396,feat_397,feat_398,feat_399,feat_400,feat_401,feat_402,feat_403,feat_404,feat_405,feat_406,feat_407,feat_408,feat_409,feat_410,feat_411,feat_412,feat_413,feat_414,feat_415,feat_416,feat_417,feat_418,feat_419,feat_420,feat_421,feat_422,feat_423,feat_424,feat_425,feat_426,feat_427,feat_428,feat_429,feat_430,feat_431,feat_432,feat_433,feat_434,feat_435,feat_436,feat_437,feat_438,feat_439,feat_440,feat_441,feat_442,feat_443,feat_444,feat_445,feat_446,feat_447,feat_448,feat_449,feat_450,feat_451,feat_452,feat_453,feat_454,feat_455,feat_

In [58]:
# Make sure the columns match exactly (except tb_status if it's not available)
X_test = df_test.drop('tb_status', axis=1, errors='ignore')  # if no tb_status yet

In [69]:
# Apply the same scaler from training
#X_test_scaled = scaler.transform(X_test)


In [70]:
#X_test = X_test[X_train.columns]

In [61]:
df_test.columns

Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9',
       ...
       'reported_cough_dur', 'tb_prior', 'tb_prior_Pul', 'hemoptysis',
       'heart_rate', 'temperature', 'weight_loss', 'smoke_lweek', 'fever',
       'night_sweats'],
      dtype='object', length=784)

In [62]:
df.columns

Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9',
       ...
       'tb_prior', 'tb_prior_Pul', 'hemoptysis', 'heart_rate', 'temperature',
       'weight_loss', 'smoke_lweek', 'fever', 'night_sweats', 'tb_status'],
      dtype='object', length=783)

In [63]:
extra_in_test = set(df_test.columns) - set(df.drop(columns='tb_status').columns)
print(extra_in_test)

{'sound_prediction_score', 'participant'}


In [64]:
X_test = df_test[df.drop(columns='tb_status').columns]

In [65]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [66]:
# Predict on test data
y_pred = model.predict(X_test)

In [67]:
# View first 10 predictions
print(y_pred[:10])

[0 0 0 0 0 0 0 0 0 0]


In [71]:
# Add predictions to test DataFrame
df_test['predicted_tb_status'] = y_pred

# Save to CSV
#df_test.to_csv('solicited_test_predictions.csv', index=False)

In [72]:


unique, counts = np.unique(y_pred, return_counts=True)
print(dict(zip(unique, counts)))

{np.int64(0): np.int64(9749)}


In [68]:
from sklearn.metrics import classification_report, confusion_matrix

# Suppose y_test is the actual TB status
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

NameError: name 'y_test' is not defined

In [73]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([ 223, 3928]))

In [74]:
from sklearn.metrics import classification_report, confusion_matrix

y_train_pred = model.predict(X_train)
print(confusion_matrix(y_train, y_train_pred))
print(classification_report(y_train, y_train_pred))

[[ 223    0]
 [   0 3928]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       223
           1       1.00      1.00      1.00      3928

    accuracy                           1.00      4151
   macro avg       1.00      1.00      1.00      4151
weighted avg       1.00      1.00      1.00      4151



In [75]:
print(df_test['tb_status'].value_counts())

KeyError: 'tb_status'

In [76]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [77]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

In [79]:
model = RandomForestClassifier(random_state=42)

In [80]:
model.fit(X_resampled, y_resampled)

SyntaxError: invalid non-printable character U+00A0 (2107256122.py, line 1)

In [81]:
with open("your_notebook_name.ipynb", encoding="utf-8") as f:
    content = f.read()

if '\u00A0' in content:
    print("Non-breaking space detected!")

FileNotFoundError: [Errno 2] No such file or directory: 'your_notebook_name.ipynb'